In [7]:
# ============================================================
# 📊 Datos de Panel: Modelo de Efectos Fijos + Animación (GIF)
# Autor: Gladys Choque Ulloa
# ============================================================

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# -----------------------------
# 1. Simulación de datos de panel
# -----------------------------
np.random.seed(42)
N_zonas = 4
units_per_zone = 5        # unidades por zona (total unidades = N_zonas * units_per_zone)
N = N_zonas * units_per_zone
T = 36                    # meses (3 años)
periods = pd.date_range(start="2020-01-01", periods=T, freq='ME')  # 'ME' evita el warning

zones = [f"Zona_{i+1}" for i in range(N_zonas)]

# Asignar unidades a cada zona
unit_zone = []
for z in zones:
    unit_zone += [z] * units_per_zone
unit_ids = [f"U{i+1}" for i in range(N)]

rows = []
for i, uid in enumerate(unit_ids):
    zone = unit_zone[i]
    alpha_i = np.random.normal(loc=2.0, scale=0.5)  # efecto individual
    temp_base = {"Zona_1": 20, "Zona_2": 22, "Zona_3": 24, "Zona_4": 26}[zone]

    for t_idx, date in enumerate(periods):
        temp = temp_base + 3 * np.sin(2 * np.pi * (t_idx / 12)) + np.random.normal(0, 1)
        y = 10 + 0.8 * temp + alpha_i + 0.5 * np.random.normal()  # variable dependiente
        rows.append([uid, zone, date, temp, y])

df = pd.DataFrame(rows, columns=["unit", "zone", "date", "temp", "y"])

# -----------------------------
# 2. Ajuste del modelo de efectos fijos (vía dummies)
# -----------------------------
X = pd.concat([df[["temp"]], pd.get_dummies(df["unit"], prefix="unit", drop_first=True)], axis=1)
X = sm.add_constant(X)
X = X.astype(float)  # ✅ conversión necesaria para evitar ValueError
model = sm.OLS(df["y"], X).fit()
df["y_pred"] = model.predict(X)

# -----------------------------
# 3. Agregar por zona y fecha
# -----------------------------
agg = df.groupby(["zone", "date"]).agg(
    y_obs=("y", "mean"),
    y_pred=("y_pred", "mean")
).reset_index()

obs_pivot = agg.pivot(index="date", columns="zone", values="y_obs")
pred_pivot = agg.pivot(index="date", columns="zone", values="y_pred")

# -----------------------------
# 4. Crear animación
# -----------------------------
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_title("Ventas promedio por zona: Observado vs Predicho (animado)")
ax.set_xlabel("Fecha")
ax.set_ylabel("Ventas promedio (y)")

lines = {}
for zone in zones:
    l_obs, = ax.plot([], [], linestyle='solid', label=f"{zone} - observado")
    l_pred, = ax.plot([], [], linestyle='dashed', label=f"{zone} - predicho")
    lines[f"{zone}_obs"] = l_obs
    lines[f"{zone}_pred"] = l_pred

ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0.)

dates = obs_pivot.index
max_frames = len(dates)

def init():
    ax.set_xlim(dates.min(), dates.max())
    ymin = min(obs_pivot.min().min(), pred_pivot.min().min()) - 1
    ymax = max(obs_pivot.max().max(), pred_pivot.max().max()) + 1
    ax.set_ylim(ymin, ymax)
    for ln in lines.values():
        ln.set_data([], [])
    return list(lines.values())

def update(frame):
    current_dates = dates[:frame + 1]
    for zone in zones:
        y_obs = obs_pivot[zone].values[:frame + 1]
        y_pred = pred_pivot[zone].values[:frame + 1]
        lines[f"{zone}_obs"].set_data(current_dates, y_obs)
        lines[f"{zone}_pred"].set_data(current_dates, y_pred)
    ax.set_title(f"Ventas promedio por zona — hasta {current_dates[-1].strftime('%Y-%m')}")
    return list(lines.values())

anim = FuncAnimation(fig, update, frames=max_frames, init_func=init, blit=True, interval=300)

# -----------------------------
# 5. Guardar en formato GIF
# -----------------------------
anim.save("panel_fit.gif", writer=PillowWriter(fps=3))
plt.close(fig)

print("✅ GIF generado y guardado como 'panel_fit.gif'")


✅ GIF generado y guardado como 'panel_fit.gif'
